In [1]:
import numpy as np
from scipy.stats import multivariate_normal

def generate_data(num_samples, mean1, cov1, mean2, cov2, mixing_coeff):
    np.random.seed(42)
    data = []
    for _ in range(num_samples):
        if np.random.rand() < mixing_coeff:
            sample = np.random.multivariate_normal(mean1, cov1)
        else:
            sample = np.random.multivariate_normal(mean2, cov2)
        data.append(sample)
    return np.array(data)

def expectation_step(data, means, covs, weights):
    num_components = len(means)
    num_samples = len(data)
    
    responsibilities = np.zeros((num_samples, num_components))
    
    for k in range(num_components):
        responsibilities[:, k] = weights[k] * multivariate_normal.pdf(data, mean=means[k], cov=covs[k])
    
    responsibilities /= np.sum(responsibilities, axis=1, keepdims=True)
    
    return responsibilities

def maximization_step(data, responsibilities):
    num_samples, num_components = responsibilities.shape
    weights = np.sum(responsibilities, axis=0) / num_samples
    
    means = np.dot(responsibilities.T, data) / np.sum(responsibilities, axis=0, keepdims=True).T
    
    covs = []
    for k in range(num_components):
        diff = data - means[k]
        cov_k = np.dot(responsibilities[:, k] * diff.T, diff) / np.sum(responsibilities[:, k])
        covs.append(cov_k)
    
    return means, covs, weights

def EM_algorithm(data, num_components, max_iters=100, tol=1e-4):
    num_samples, num_features = data.shape
    
    means = np.random.rand(num_components, num_features)
    covs = [np.identity(num_features) for _ in range(num_components)]
    weights = np.ones(num_components) / num_components
    
    for _ in range(max_iters):
        responsibilities = expectation_step(data, means, covs, weights)
        
        new_means, new_covs, new_weights = maximization_step(data, responsibilities)
        
        mean_diff = np.linalg.norm(new_means - means)
        cov_diff = sum(np.linalg.norm(new_cov - cov) for new_cov, cov in zip(new_covs, covs))
        weight_diff = np.linalg.norm(new_weights - weights)
        
        if mean_diff < tol and cov_diff < tol and weight_diff < tol:
            break
        
        means, covs, weights = new_means, new_covs, new_weights
    
    return means, covs, weights

num_samples = 1000
mean1 = np.array([2, 2])
cov1 = np.array([[1, 0.5], [0.5, 1]])
mean2 = np.array([-2, -2])
cov2 = np.array([[1, -0.5], [-0.5, 1]])
mixing_coeff = 0.5

data = generate_data(num_samples, mean1, cov1, mean2, cov2, mixing_coeff)

num_components = 2
estimated_means, estimated_covs, estimated_weights = EM_algorithm(data, num_components)

print("True Means:")
print(f"Component 1: {mean1}")
print(f"Component 2: {mean2}")

print("\nEstimated Means:")
for i, mean in enumerate(estimated_means):
    print(f"Component {i+1}: {mean}")

print("\nTrue Covariances:")
print(f"Component 1:\n{cov1}")
print(f"Component 2:\n{cov2}")

print("\nEstimated Covariances:")
for i, cov in enumerate(estimated_covs):
    print(f"Component {i+1}:\n{cov}")

print("\nTrue Mixing Coefficient:")
print(mixing_coeff)

print("\nEstimated Mixing Coefficients:")
print(estimated_weights)

True Means:
Component 1: [2 2]
Component 2: [-2 -2]

Estimated Means:
Component 1: [1.97081956 2.05009926]
Component 2: [-1.90435638 -2.03625316]

True Covariances:
Component 1:
[[1.  0.5]
 [0.5 1. ]]
Component 2:
[[ 1.  -0.5]
 [-0.5  1. ]]

Estimated Covariances:
Component 1:
[[1.05571296 0.54198591]
 [0.54198591 0.98257125]]
Component 2:
[[ 0.87754498 -0.47917373]
 [-0.47917373  1.00039894]]

True Mixing Coefficient:
0.5

Estimated Mixing Coefficients:
[0.49892975 0.50107025]
